In [1]:
import advent
from advent.intcode import BaseIO
import time
import numpy as np
from IPython.display import clear_output

data = advent.get_intcode(15)

In [2]:
class IO(BaseIO):

    pieces = {0: 'X', 1: '.', 2: '!', 3: 'o', 4: '?'}

    def __init__(self):
        self.output = 1
        self.maze = {(0, 0): 1}
        self.position = (0, 0)
        self.direction = (0, 0)
    
    def print(self):
        min_x = min(self.maze.keys(), key=lambda a: a[1])[1]
        max_x = max(self.maze.keys(), key=lambda a: a[1])[1]
        min_y = min(self.maze.keys(), key=lambda a: a[0])[0]
        max_y = max(self.maze.keys(), key=lambda a: a[0])[0]
        grid = np.ones((max_y-min_y+1, max_x-min_x+1)) * 4
        for k in self.maze:
            a, b = k
            grid[a-min_y, b-min_x] = self.maze[k]
        grid[self.position[0]-min_y, self.position[1]-min_x] = 3
        output = [''.join(map(lambda x: self.pieces[x], line)) for line in grid]
        print('\n'.join(output), flush=True)
    
    def handle_output(self):
        # Position is the 'old' position
        if self.output == 1: # empty spot
            self.position = (self.position[0] + self.direction[0], self.position[1] + self.direction[1])
            self.maze[self.position] = 1
        elif self.output == 2:
            self.position = (self.position[0] + self.direction[0], self.position[1] + self.direction[1])
            self.maze[self.position] = 2
        elif self.output == 0:
            tmp_position = (self.position[0] + self.direction[0], self.position[1] + self.direction[1])
            self.maze[tmp_position] = 0

    def read(self):
        self.handle_output()
        clear_output(wait=True)
        self.print()
        time.sleep(0.1)
        result = None
        while result not in ['1', '2', '3', '4', 'w', 'a', 's', 'd']:
            result = input("Your input? (1=north, 2=south, 3=west, 4=east)")
        if result in ['w', 'a', 's', 'd']: result = {'w': 1, 'a': 3, 's': 2, 'd': 4}[result]
        self.direction = {1: (-1, 0), 2: (1, 0), 3: (0, -1), 4: (0, 1)}[int(result)]
        return int(result)

    def write(self, value):
        self.output = value
        return self

    def output(self):
        return self.maze

In [3]:
data = advent.get_intcode(15)
io = IO()

# _, out = run(data.copy(), io)
# I would use python maze.py instead See the explored maze at data/maze.txt (yes I did it by hand! :D)

In [4]:
maze_char = advent.get_lines('maze')
# Y is manually added and means unreachable, so should never come up anyway
# ' ' is oxygen
maze = {}
for y in range(len(maze_char)):
    for x in range(len(maze_char[y])):
        c = maze_char[y][x]
        c = '.' if c == '?' else c
        c = ' ' if c == '!' else c
        maze[(y, x)] = c

counter = 0
while '.' in maze.values():
    counter += 1
    maze_old = maze.copy()
    for k in maze:
        if maze_old[k] == ' ':
            y, x = k
            if maze_old[(y-1, x)] != 'X': maze[(y-1, x)] = ' '
            if maze_old[(y+1, x)] != 'X': maze[(y+1, x)] = ' '
            if maze_old[(y, x-1)] != 'X': maze[(y, x-1)] = ' '
            if maze_old[(y, x+1)] != 'X': maze[(y, x+1)] = ' '

counter

288

In [5]:
# Example of using advent.maze.solve_maze:
from advent.maze import solve_maze
import advent

data = advent.get_lines('maze')

maze = {}
for y in range(len(data)):
    for x in range(len(data[y])):
        maze[(y, x)] = data[y][x]
        if maze[(y, x)] == 's': start = (y, x)

print("Start:", start)

def adjacent(n):
    nodes = [(n[0]+1, n[1]), (n[0]-1, n[1]), (n[0], n[1]+1), (n[0], n[1]-1)]
    return [(n, 1) for n in nodes if maze[n] != 'X'] # cost is just 1

def is_target(n):
    return maze[n] == '!'

w, parents = solve_maze(start, is_target, adjacent, total_nodes = sum([x == '.' for x in maze.values()]))
# should be same as part 1, which I previously MANUALLY COUNTED :D, so 230
# yeah this probably would have been faster...

Start: (21, 21)


  0%|          | 0/1 [00:00<?, ?it/s]

Final path length: 230
